In [44]:
import pandas as pd
import numpy as np
import math
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, CrosshairTool, HoverTool, NumeralTickFormatter
from bokeh.plotting import figure
from bokeh.layouts import column, row
from bokeh.transform import cumsum
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm

stock_list = pd.read_csv('../input/stock_list.csv')

returns = pd.DataFrame()
last_price = np.zeros((len(stock_list)))

cleancomma = lambda items: float(str(items).replace(',', ''))
to_float = lambda items: float(items)

for i in range(len(stock_list)):
    stock_name = stock_list.stock_name[i]
    print(stock_name)
    path = '../input/'+stock_name+'.csv'
    stock = pd.read_csv(path)
    stock['f_adj_price'] = stock['Adj Close'].map(cleancomma).map(to_float)
    returns[stock_name] = (stock['f_adj_price']/stock['f_adj_price'].shift(12))-1
    last_price[i] = stock['f_adj_price'].iloc[-1]

ASHOKLEY.BO
BAJAJ-AUTO.NS
BEL.BO
BHEL.BO
BPCL.NS
CASTROLIND.NS
CESC.NS
COLPAL.NS
CUMMINSIND.NS
CYIENT.NS
EMAMILTD.NS
ENGINERSIN.NS
GAIL.NS
HEROMOTOCO.NS
HINDPETRO.NS
HINDUNILVR.NS
IOC.NS
INFY.NS
ITC.NS
MGL.NS
MINDTREE.NS
MPHASIS.NS
NESTLEIND.NS
NHPC.NS
NTPC.NS
ONGC.NS
OIL.NS
PETRONET.NS
POWERGRID.NS
RITES.NS
STAR.NS
SUNTV.NS
TCS.NS
TATAPOWER.NS
TECHM.NS
TORNTPOWER.NS
SBIN.NS
SBIGETS.BO


In [45]:
cov_mat = returns.dropna().cov()

In [ ]:
num_port = 30000
rf = 0.04
all_wts = np.zeros((num_port, len(returns.columns)))
port_returns = np.zeros((num_port))
port_risk = np.ones((num_port))
sharpe_ratio = np.zeros((num_port))

# SIMULATE x PORTFOLIOS

np.random.seed(42)

for i in range(num_port):

    # Portfolio weights
    wts = np.random.uniform(size=len(returns.columns))
    wts = wts / np.sum(wts)
    all_wts[i, :] = wts

    # Portfolio Return
    port_ret = np.sum(returns.mean() * wts)
    port_returns[i] = port_ret

    # Portfolio Risk
    port_sd = np.sqrt(np.dot(wts.T, np.dot(cov_mat, wts)))
    port_risk[i] = port_sd

    # Portfolio Sharpe Ratio, assuming 0% Risk Free Rate
    sr = (port_ret - rf) / port_sd
    sharpe_ratio[i] = sr

    # Save portfolios of interest (min var, max return and max SR)
    if sr >= max(sharpe_ratio[0:-1]):
        max_sr_ret = port_ret
        max_sr_risk = port_sd
        max_sr_w = wts
        max_sr = sr

    if port_ret >= max(port_returns[0:-1]):
        max_ret_ret = port_ret
        max_ret_risk = port_sd
        max_ret_w = wts

    if port_sd <= min(port_risk[0:-1]):
        min_var_ret = port_ret
        min_var_risk = port_sd
        min_var_w = wts

In [ ]:
p = figure(plot_height=700, plot_width=770, title="Efficient frontier. Simulations: " + str(num_port),
           tools='box_zoom,wheel_zoom,reset', toolbar_location='above')
p.add_tools(CrosshairTool(line_alpha=1, line_color='lightgray', line_width=1))
p.add_tools(HoverTool(tooltips=None))
source = ColumnDataSource(data=dict(risk=port_risk, profit=port_returns))
p.circle(x='risk', y='profit', source=source, line_alpha=0, hover_color='navy', alpha=0.4, hover_alpha=1, size=8)
p.circle(min_var_risk, min_var_ret, color='lime', legend_label='Portfolio with minimum variance', size=10)
p.circle(max_sr_risk, max_sr_ret, color='orangered', legend_label='Portfolio with max Sharpe ratio', size=12)
p.circle(max_ret_risk, max_ret_ret, color='olive', legend_label='Portfolio with max return', size=9)
p.legend.location = "bottom_right"
p.xaxis.axis_label = 'Volatility, or risk (standard deviation)'
p.yaxis.axis_label = 'Annual return'
p.xaxis[0].formatter = NumeralTickFormatter(format="0.0%")
p.yaxis[0].formatter = NumeralTickFormatter(format="0.0%")

In [ ]:
#Portfolio composition. Min variance, max SR, max return
"""
def plot_portfolio_composition(ticks, weights, plot_name):

    x = dict()
    for i in range(len(ticks)):
        x[ticks[i]] = weights[i]

    #color_list = [ "olive", "yellowgreen", "lime", "chartreuse", "springgreen", "lightgreen", "darkseagreen",
                   #"seagreen", "green", "darkgreen"]

    plot_data = pd.Series(x).reset_index(name='value').rename(columns={'index': 'stock'})
    plot_data['angle'] = plot_data['value'] / plot_data['value'].sum() * 2 * math.pi
    #plot_data['color'] = color_list[0:len(weights)]
    p = figure(plot_height=250, plot_width=250, title=plot_name, toolbar_location=None,
                      tools="hover", tooltips="@stock: @value{%0.1f}", x_range=(-0.5, 1.0))
    p.wedge(x=0, y=1, radius=0.4, start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
                   line_color="white", color='color', source=plot_data)
    p.axis.axis_label = None
    p.axis.visible = False
    p.grid.grid_line_color = None
    p.outline_line_color = None

    return p

p_minvar = plot_portfolio_composition(stock_list.stock_name, min_var_w, "Portfolio with minimum variance")
p_maxsr = plot_portfolio_composition(stock_list.stock_name, max_sr_w, "Portfolio with max Sharpe ratio")
p_maxret = plot_portfolio_composition(stock_list.stock_name, max_ret_w, "Portfolio with max return")
"""

In [ ]:
#layout = row([p_minvar, p_maxsr, p_maxret])
#show(layout)

In [ ]:
show(p)

In [ ]:
wts = max_sr_w

In [ ]:
output_data = {'stocks': stock_list.stock_name, 'Weights': wts}

In [ ]:
print(pd.DataFrame(output_data))

In [ ]:
sum(wts)

In [ ]:
ret = np.sum(returns.mean() * wts)
sd = np.sqrt(np.dot(wts.T, np.dot(cov_mat, wts)))
ret, sd

In [ ]:
for col in returns.columns:
    
    sns.distplot(returns[col])
    plt.show()

In [ ]:
for col in returns.columns:
    
    print("For stock %s, returns is %0.4f" %(col, returns[col].mean()))

In [ ]:
amt = 300000
qty = (amt * wts)/ last_price
qtyf = [int(i) for i in qty]

In [ ]:
output_data = {'stocks': stock_list.stock_name, 'Share': qtyf}
print(pd.DataFrame(output_data))

In [ ]:
conf_level1 = 0.0001

mean_investment = (1+ret) * amt
stdev_investment = amt * sd

cutoff1 = norm.ppf(conf_level1, mean_investment, stdev_investment)

var_1yr = amt - cutoff1
var_1yr

In [ ]:
for i in range (2, 5):
    print(np.round(var_1yr * np.sqrt(i)))